# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы
## Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

BPR - https://www.gabormelli.com/RKB/Bayesian_Personalized_Ranking_(BPR)_Algorithm, <br>https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9

WARP - https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

---

In [1]:
# !pip install lightfm
# !pip install hyperopt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# LightFM
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Libraries for working with paths and saving
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k as custom_precision, recall_at_k
from src.utils import prefilter_items
# from recommenders import MainRecommender

from hyperopt import fmin, tpe, hp
from functools import partial

In [3]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# Используем train-test split по времени, а не случайно. Возьмем последние 3 недели в качестве теста

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [6]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [7]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [8]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## 1. Filter items

In [10]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


## 2.1 Prepare csr train matrix

In [11]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,42346,42521,78986,108646,117847,244960,818981,819048,819255,819308,...,12949855,12988031,13002975,13003092,13003094,13039088,13071902,13072793,13158064,13158992
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [12]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [14]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

# user_feat.head(2)
# item_feat.head(2)

In [15]:
user_feat.shape, item_feat.shape

((2484, 7), (5001, 6))

## 4. Encoding features

In [16]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
item_feat_lightfm.head(2)

,manufacturer_2.0,manufacturer_5.0,manufacturer_16.0,manufacturer_20.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_43.0,manufacturer_61.0,manufacturer_67.0,...,curr_size_of_product_L 32OZ,curr_size_of_product_L 16 OZ,curr_size_of_product_L 7.75 OZ,curr_size_of_product_L 13.25 OZ,curr_size_of_product_LITER,curr_size_of_product_N 12 OZ,curr_size_of_product_PINT,curr_size_of_product_PK,curr_size_of_product_PT,curr_size_of_product_QT
item_id,,,,,,,,,,,,,,,,,,,,,
42346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 5. Init model

In [18]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","warp"
                learning_rate=0.01, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42,
                # k=5,
                # n=15,
                # max_sampled=100
               )

## 6. Train

In [19]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

## 7. Getting embeddings

In [20]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [21]:
user_emb[0]  # biases

array([-2.0735104,  0.       ,  0.       , ..., -2.1883667, -1.6312253,
        0.       ], dtype=float32)

In [22]:
user_emb[0].shape

(2484,)

In [23]:
user_emb[1]  # embeddings

array([[ 1.34870415e-05, -6.08253868e-06, -1.99232909e-05, ...,
        -6.18958575e-06,  2.88295087e-06,  4.18105355e-06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.71565262e-05, -7.90166541e-06, -1.46653510e-05, ...,
        -8.46447074e-06,  4.33392597e-06,  6.35113838e-06],
       [ 9.92816422e-06, -5.66551807e-06, -1.12894959e-05, ...,
        -9.73420356e-07,  8.03464263e-06, -7.05138461e-07],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [24]:
user_emb[1].shape

(2484, 40)

## 8. Вектора по товарам

In [25]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [26]:
item_emb[0].shape

(5001,)

In [27]:
item_emb[1].shape

(5001, 40)

## 9. Evaluation -> Train precision

In [28]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.19758455

In [29]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

## 10. Predict

In [30]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [31]:
users_ids_row[:10]

array([2358, 1358, 1358, 1358, 1358, 1166, 1166, 1166, 1166, 1166])

In [32]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [33]:
# добавляем наш полученный скор в трейн датафрейм
data_train['score'] = predictions

In [34]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,score
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99,-0.392412
11,1364,26984896261,1,999999,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19,-1.723243
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99,-1.723243
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09,-1.723243
14,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50,-2.042611


In [35]:
# создаем предикт датафрейм в формате списка това
predict_result = data_train[['user_id','item_id','score']][data_train.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [36]:
predict_result.head()

,user_id,item_id
0,1,"[1029743, 1055831, 6034857, 974327, 952163, 10..."
1,2,"[1106523, 1075368, 969725, 1053107, 952163, 90..."
2,3,"[1106523, 983584, 940375, 946839, 839656, 1050..."
3,4,"[1029743, 1075368, 1052294, 1093697, 1056778, ..."
4,5,"[1126899, 1029743, 6034991, 825659, 835986, 92..."


In [37]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [38]:
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 1055831, 6034857, 974327, 952163, 10..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 983584, 940375, 946839, 839656, 1050..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1098167, 1139835, 904997, 9..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 7147142, 9835880, 1077373, 1122358, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 6919458, 1139835, 1093329, ..."


## 11. Test with custom precision func

In [39]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.12861180382377269


In [40]:
precision = df_result_for_metrics.apply(lambda row: recall_at_k(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.016970250963808994


##  12. Grid Search

In [41]:
def objective(params, user_item_matrix, user_feat_lightfm, item_feat_lightfm, test_user_item_matrix):
    
    model = LightFM(no_components=int(params['no_components']),
                loss=params['loss'],
                learning_rate=params['learning_rate'], 
                item_alpha=params['item_alpha'],
                user_alpha=params['user_alpha'], 
                random_state=42)
    
    model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=int(params['epochs']), 
          num_threads=4,
          verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    return -test_precision

In [50]:
search_space = {"no_components": hp.quniform('no_components', 10, 60, 1), "loss": hp.choice('loss', ['bpr', 'warp']), 
                "learning_rate": hp.uniform('learning_rate', 0.01, 0.1), "item_alpha": hp.uniform('item_alpha', 0.05, 0.1),
                "user_alpha": hp.uniform('user_alpha', 0.05, 0.1), "epochs": hp.quniform('epochs', 10, 30, 1)}

In [51]:
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, user_item_matrix=user_item_matrix, user_feat_lightfm=user_feat_lightfm, 
                       item_feat_lightfm=item_feat_lightfm, test_user_item_matrix=test_user_item_matrix),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций 
            max_evals=5, # max_evals=100,
          # progressbar
            show_progressbar=True
        )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:18<00:00, 15.62s/trial, best loss: -0.00451918039470911]


In [52]:
print(best)

{'epochs': 29.0, 'item_alpha': 0.06766203244170595, 'learning_rate': 0.08319282674822862, 'loss': 0, 'no_components': 35.0, 'user_alpha': 0.09512275113063665}
